This notebook is to create a network including the nodes and links from Gijs' "werkgevers aansprakelijkheid" network.

In [1]:
from SPARQLWrapper import SPARQLWrapper, JSON
import os
import sys
sys.path.insert(0, os.path.abspath('..'))
import  parser.matcher as matcher
from blazegraph_querier import query_to_json

In [2]:
sparql = SPARQLWrapper("http://localhost:9999/blazegraph/namespace/hogeraad/sparql")

In [3]:
import pandas as pd
filepath_csv = '/media/sf_VBox_Shared/CaseLaw/graphs/test3 wg aanspr.csv'
links_wgaans = pd.read_csv(filepath_csv, sep=';', header=0)
links_wgaans.columns = ['id', 'reference']
links_wgaans = links_wgaans.drop_duplicates()

In [4]:
links_wgaans = links_wgaans.applymap(lambda ecli: 'http://deeplink.rechtspraak.nl/uitspraak?id='+ecli)

In [35]:
links_wgaans.head()

,id,reference
0,http://deeplink.rechtspraak.nl/uitspraak?id=EC...,http://deeplink.rechtspraak.nl/uitspraak?id=EC...
1,http://deeplink.rechtspraak.nl/uitspraak?id=EC...,http://deeplink.rechtspraak.nl/uitspraak?id=EC...
2,http://deeplink.rechtspraak.nl/uitspraak?id=EC...,http://deeplink.rechtspraak.nl/uitspraak?id=EC...
3,http://deeplink.rechtspraak.nl/uitspraak?id=EC...,http://deeplink.rechtspraak.nl/uitspraak?id=EC...
4,http://deeplink.rechtspraak.nl/uitspraak?id=EC...,http://deeplink.rechtspraak.nl/uitspraak?id=EC...


In [5]:
ids_wgaans = list(pd.concat([links_wgaans['id'], links_wgaans['reference']]).unique())

In [6]:
ids_wgaans[0]

'http://deeplink.rechtspraak.nl/uitspraak?id=ECLI:NL:HR:2012:BV1295'

In [7]:
result = []
for ecli in ids_wgaans:
    querystring = '''
            prefix dcterm: <http://purl.org/dc/terms/>
            select ?type ?id ?to ?title ?creator ?date ?subject ?abstract ?hasVersion ?article
            where {
                {
                BIND("node" AS ?type).
                ?id dcterm:type	<http://psi.rechtspraak.nl/uitspraak>.
                filter (?id = <''' + ecli+'''>).
                optional { ?id dcterm:creator ?creator}.
                   optional { ?id dcterm:abstract ?abstract}.
                   optional { ?id dcterm:subject ?subject}.
                   optional { ?id dcterm:date ?date}.
                   optional { ?id dcterm:title ?title}
                }
                
                union
                {
                BIND("vindplaats" AS ?type).
                ?id dcterm:hasVersion ?hasVersion
                filter (?id = <''' + ecli+'''>).
                }
                union
                  {
                    BIND("article" AS ?type).
                    ?id dcterm:reference ?articleid .
                    ?articleid rdfs:label "Wetsverwijzing".
                    ?articleid dcterm:title ?article
                    filter (?id = <''' + ecli+'''>).
                  }
              }
            '''
    #print(querystring)
    sparql.setQuery(querystring)
    sparql.setReturnFormat(JSON)
    ret = sparql.query()
    result.append(ret.convert())

In [8]:
variables = [x for x in result[0]['head']['vars'] if
                 x not in ['type', 'from', 'to', 'hasVersion', 'article']]

In [21]:
node_list = [res for node in result for res in node['results']['bindings'] ]

In [22]:
nodes = [res for res in node_list if res['type']['value']=='node']

In [23]:
vindplaatsen = [res for res in node_list if
                    res['type']['value'] == 'vindplaats']
articles = [res for res in node_list if
                res['type']['value'] == 'article']

In [24]:
nodes_json, node_ids = query_to_json.parse_nodes(nodes, variables)

In [25]:
# Add nodes for the empty nodes
empty_node = {var: '' for var in variables if var != 'id'}
node_ids = [node['id'] for node in nodes_json]

for ecli in ids_wgaans:
    id0 = ecli
    if id0 not in node_ids:
        nodes_json.append({'id':id0, **empty_node})

In [26]:
nodes_json = query_to_json.enrich_nodes(nodes_json, vindplaatsen, articles)

In [38]:
links = [{'source': row['id'], 'target': row['reference'], 'id': row['id']+'_'+row['reference']} 
         for ind, row in links_wgaans.iterrows()]

In [28]:
nodes_json = query_to_json.network_analysis.add_network_statistics(nodes_json, links)

In [29]:
len(nodes_json)

156

In [34]:
print(len(ids_wgaans), len(nodes_json))

156 156


In [42]:
nodes_json[0]

{'abstract': 'Uitleg dekkingsomvang AVB-polis. Aansprakelijkheid werkgever bij verkeersongeval werknemer op grond van art. 7:611 BW (verzekeringsplicht niet nagekomen). Is schade werknemer letselschade in zin van polis? Functie AVB-polis en daarop gebaseerde verwachtingen rechtvaardigen ruime uitleg dekkingsomvang; dekking tegen aansprakelijkheid voor letselschade omvat in beginsel ook dekking tegen op art. 7:611 gebaseerde aansprakelijkheid.',
 'articles': ['7:611 bw',
  'Burgerlijk Wetboek Boek 7',
  'Burgerlijk Wetboek Boek 7 685',
  'Burgerlijk Wetboek Boek 7 611'],
 'authorities': 0.0,
 'betweenness_centrality': 0.0,
 'count_annotation': 2,
 'count_version': 8,
 'creator': 'http://standaarden.overheid.nl/owms/terms/Hoge_Raad_der_Nederlanden',
 'date': '2012-03-30',
 'degree': 1,
 'degree_centrality': 0.0064516129032258064,
 'ecli': 'ECLI:NL:HR:2012:BV1295',
 'hubs': 4.5281207035507266e-97,
 'id': 'http://deeplink.rechtspraak.nl/uitspraak?id=ECLI:NL:HR:2012:BV1295',
 'in_degree': 0

In [40]:
links[0]

{'id': 'http://deeplink.rechtspraak.nl/uitspraak?id=ECLI:NL:HR:2012:BV1295_http://deeplink.rechtspraak.nl/uitspraak?id=ECLI:NL:HR:2011:BR5215',
 'source': 'http://deeplink.rechtspraak.nl/uitspraak?id=ECLI:NL:HR:2012:BV1295',
 'target': 'http://deeplink.rechtspraak.nl/uitspraak?id=ECLI:NL:HR:2011:BR5215'}

In [41]:
import json

filename = '/media/sf_VBox_Shared/CaseLaw/graphs/json_sigma/wgaans.json'

with open(filename, 'w') as outfile:
    json.dump({'nodes': nodes_json, 'edges': links}, fp=outfile)

In [33]:
import json

filename = '/media/sf_VBox_Shared/CaseLaw/graphs/d3_wgaans.json'

with open(filename, 'w') as outfile:
    json.dump({'nodes': nodes_json, 'links': links}, fp=outfile)